<a href="https://colab.research.google.com/github/salunkheketki19/Pytorch-Scholarship/blob/master/Image_Classifier_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
file_id = 'Enter yout file id' # URL id. 
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('flower_data.zip')

In [0]:
!unzip flower_data.zip 

In [0]:
# !pip install torchvision
import torch
from torchvision import datasets, transforms, models
from torch import nn
from torch import optim
import torch.nn.functional as F

In [0]:
data_dir = './flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

In [0]:
# TODO: Define your transforms for the training and validation sets
data_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])
valid_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder
image_datasets = datasets.ImageFolder(train_dir, transform= data_transforms)
valid_datasets = datasets.ImageFolder(valid_dir, transform= valid_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
train_loaders = torch.utils.data.DataLoader(image_datasets, batch_size=64, shuffle=True)
valid_loaders = torch.utils.data.DataLoader(valid_datasets, batch_size=64, shuffle=True)

In [0]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [0]:
# TODO: Build and train your network
model = models.densenet121(pretrained=True)
model

In [0]:
for param in model.parameters():
    param.requires_grad = False
    
from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                           ('fc1', nn.Linear(1024,500)),
                           ('relu', nn.ReLU()),
                           ('dropout', nn.Dropout(0.2)),
                           ('fc2', nn.Linear(500,102)),
                           ('output', nn.LogSoftmax(dim=1))
                            ]))
model.classifier = classifier
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [0]:
import time 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
model.to(device)

In [0]:
epochs = 15
steps = 0
running_loss = 0
print_every = 50

for epoch in range(epochs):
  for inputs, labels in train_loaders:
    steps += 1
    inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()
    output = model.forward(inputs)
    loss = criterion(output, labels)
    loss.backward()
    optimizer.step()
    
    running_loss +=  loss.item()
    
    if steps % print_every == 0:
        valid_loss = 0
        accuracy = 0
        model.eval()
        with torch.no_grad():
            for inputs, labels in valid_loaders:
                inputs, labels = inputs.to(device), labels.to(device)
                output = model.forward(inputs)
                batch_loss = criterion(output, labels)
                
                valid_loss += batch_loss
                
                ps = torch.exp(output)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
        
        print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Val loss: {valid_loss/len(valid_loaders):.3f}.. "
                  f"Val accuracy: {accuracy/len(valid_loaders):.3f}")
        running_loss = 0
        model.train()

Epoch 1/15.. Train loss: 3.916.. Val loss: 2.967.. Val accuracy: 0.370
Epoch 1/15.. Train loss: 2.394.. Val loss: 1.679.. Val accuracy: 0.667
Epoch 2/15.. Train loss: 1.532.. Val loss: 1.225.. Val accuracy: 0.715
Epoch 2/15.. Train loss: 1.147.. Val loss: 0.931.. Val accuracy: 0.798
Epoch 3/15.. Train loss: 1.004.. Val loss: 0.841.. Val accuracy: 0.792
Epoch 3/15.. Train loss: 0.833.. Val loss: 0.699.. Val accuracy: 0.835
Epoch 4/15.. Train loss: 0.755.. Val loss: 0.701.. Val accuracy: 0.821
Epoch 4/15.. Train loss: 0.744.. Val loss: 0.596.. Val accuracy: 0.847
Epoch 5/15.. Train loss: 0.612.. Val loss: 0.565.. Val accuracy: 0.867
Epoch 5/15.. Train loss: 0.634.. Val loss: 0.562.. Val accuracy: 0.873
Epoch 6/15.. Train loss: 0.608.. Val loss: 0.547.. Val accuracy: 0.841
Epoch 6/15.. Train loss: 0.558.. Val loss: 0.529.. Val accuracy: 0.880
Epoch 7/15.. Train loss: 0.586.. Val loss: 0.584.. Val accuracy: 0.853
Epoch 7/15.. Train loss: 0.510.. Val loss: 0.518.. Val accuracy: 0.871
Epoch 

In [0]:
# TODO: Save the checkpoint 
model.class_to_idx = image_datasets.class_to_idx
checkpoint = {
         'class_to_idx': model.class_to_idx,
         'state_dict': model.state_dict()
}
torch.save(checkpoint,'checkpoint.pt')
